# Deep Q-Learning

In this notebook, you will implement a deep Q-Learning reinforcement algorithm. The implementation borrows ideas from both the original DeepMind Nature paper and the more recent asynchronous version:<br/>
[1] "Human-Level Control through Deep Reinforcement Learning" by Mnih et al. 2015<br/>
[2] "Asynchronous Methods for Deep Reinforcement Learning" by Mnih et al. 2016.<br/>

In particular:
* We use separate target and Q-functions estimators with periodic updates to the target estimator. 
* We use several concurrent "threads" rather than experience replay to generate less biased gradient updates. 
* Threads are actually synchronized so we start each one at a random number of moves.
* We use an epsilon-greedy policy that blends random moves with policy moves.
* We taper the random action parameter (epsilon) and the learning rate to zero during training.

This gives a simple and reasonably fast general-purpose RL algorithm. We use it here for the Cartpole environment from OpenAI Gym, but it can easily be adapted to others. For this notebook, you will implement 4 steps:

1. The backward step for the Q-estimator
2. The $\epsilon$-greedy policy
3. "asynchronous" initialization 
4. The Q-learning algorithm

To get started, we import some prerequisites.

In [1]:
%matplotlib inline

import gym
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
import pickle

The block below lists some parameters you can tune. They should be self-explanatory. They are currently set to train CartPole-V0 to a "solved" score (> 195) most of the time. 

In [2]:
nsteps = 10001                       # Number of steps to run (game actions per environment)
npar = 16                            # Number of parallel environments
target_window = 200                  # Interval to update target estimator from q-estimator
discount_factor = 0.99               # Reward discount factor
printsteps = 1000                    # Number of steps between printouts
render = False                       # Whether to render an environment while training

epsilon_start = 1.0                  # Parameters for epsilon-greedy policy: initial epsilon
epsilon_end = 0.0                    # Final epsilon
neps = int(0.8*nsteps)               # Number of steps to decay epsilon

learning_rate = 2e-3                 # Initial learning rate
lr_end = 0                           # Final learning rate
nlr = neps                           # Steps to decay learning rate
decay_rate = 0.99                    # Decay factor for RMSProp 

nhidden = 200                        # Number of hidden layers for estimators

init_moves = 2000                    # Upper bound on random number of moves to take initially
nwindow = 2                          # Sensing window = last n images in a state

Below are environment-specific parameters. The function "preprocess" should process an observation returned by the environment into a vector for training. For CartPole we simply append a 1 to implement bias in the first layer. 

For visual environments you would typically crop, downsample to 80x80, set color to a single bit (foreground/background), and flatten to a vector. That transformation is already implemented in the Policy Gradient code.

*nfeats* is the dimension of the vector output by *preprocess*.

In [3]:
game_type="CartPole-v0"                 # Model type and action definitions
VALID_ACTIONS = [0, 1]
nactions = len(VALID_ACTIONS)
nfeats = 5                              # There are four state features plus the constant we add

def preprocess(I):                      # preprocess each observation
    """Just append a 1 to the end"""
    return np.append(I.astype(float),1) # Add a constant feature for bias

Here is the Q-estimator class. We use two instances of this class, one for the target estimator, and one for the Q-estimator. The Q function is normally represented as a scalar $Q(x,a)$ where $x$ is the state and $a$ is an action. For ease of implementation, we actually estimate a vector-valued function $Q(x,.)$ which returns the estimated reward for every action. The model here has just a single hidden layer:

<pre>
Input Layer (nfeats) => FC Layer => RELU => FC Layer => Output (naction values)
</pre>

## 1. Implement Q-estimator gradient
Your first task is to implement the
<pre>Estimator.gradient(s, a, y)</pre>
method for this class. **gradient** should compute the gradients wrt weight arrays W1 and W2 into
<pre>self.grad['W1']
self.grad['W2']</pre>
respectively. Both <code>a</code> and <code>y</code> are vectors. Be sure to update only the output layer weights corresponding to the given action vector. 

In [4]:
from __future__ import division

In [5]:
class Estimator():

    def __init__(self, ninputs, nhidden, nactions):
        """ Create model matrices, and gradient and squared gradient buffers"""
        model = {}
        model['W1'] = np.random.randn(nhidden, ninputs) / np.sqrt(ninputs)   # "Xavier" initialization
        model['W2'] = np.random.randn(nactions, nhidden) / np.sqrt(nhidden)
        self.model = model
        self.grad = { k : np.zeros_like(v) for k,v in model.iteritems() }
        self.gradsq = { k : np.zeros_like(v) for k,v in model.iteritems() }   
        

    def forward(self, s):
        """ Run the model forward given a state as input.
    returns action predictions and the hidden state"""
        h = np.dot(self.model['W1'], s) # nhidden x npars
        h[h<0] = 0 # ReLU nonlinearity # nhidden x npars
        rew = np.dot(self.model['W2'], h) # nactions, npars
        print(type(rew))
        return rew, h
    
    
    def predict(self, s):
        """ Predict the action rewards from a given input state"""
        rew, h = self.forward(s)
        return rew, h
    
              
    def gradient(self, s, a, y):
        """ Given a state s, action a and target y, compute the model gradients"""
    ##################################################################################
    ##                                                                              ##
    ## TODO: Compute gradients and return a scalar loss on a minibatch of size npar ##
    ##    s is the input state matrix (ninputs x npar).                             ##
    ##    a is an action vector (npar,).                                            ##
    ##    y is a vector of target values (npar,) corresponding to those actions.    ##
    ##    return: the loss per sample (npar,).                                      ##                          
    ##                                                                              ##
    ## Notes:                                                                       ##
    ##    * If the action is ai in [0,...,nactions-1], backprop only through the    ##
    ##      ai'th output.                                                           ##
    ##    * loss should be L2, and we recommend you normalize it to a per-input     ##
    ##      value, i.e. return L2(target,predition)/sqrt(npar).                     ##
    ##    * save the gradients in self.grad['W1'] and self.grad['W2'].              ##
    ##    * update self.grad['W1'] and self.grad['W2'] by adding the gradients, so  ##
    ##      that multiple gradient steps can be used beteween updates.              ##
    ##                                                                              ##
    ##################################################################################
        ninputs, npar = s.shape
        loss_obs = [] # Collecting L_i
        loss = 0.0
        # Calculating Loss
        # Prediction: nactions x npar, h: nhidden x npar
        prediction, h = self.predict(s) 
        nactions, _ = prediction.shape
        predicted_reward = [] 
        #w2_grads = []
        w1_grads = []
        # ----- Lambda Function to determine the positivity of deviation ---- #
        sign_func = lambda a: (a>0) - (a<0)
        self.grad['W1'] = np.zeros_like(self.model['W1'])
        self.grad['W2'] = np.zeros_like(self.model['W2'])
        
        dh = np.zeros_like(h)
        for i in range(npar):
            sign = sign_func(prediction[a[i],i]-y[i])
            loss_i = prediction[a[i], i] - y[i]
            loss_i = sign*loss_i
            self.grad['W2'][a[i], :] += sign*h[i, :].T
            print(dh)
            dh[:, i] = sign*(self.model['W2'][a[i],:]).T
            loss += loss_i
        
        self.grad['W1'] = np.dot(dh.T, s)
            
            
            
            
            
            
            
            # For each environment, we are conducting backpropagation
            #dscores = np.zeros(nactions)
            #dscores[a[i]] = 1
            #square_diffs[i] = (prediction[a[i], i] - y[i])**2
            #one_hot = np.zeros(nactions)
            #one_hot[a[i]] = 2*(prediction[a[i], i] - y[i]) # nactions x 1
            #tmp = np.zeros_like(self.model['W2'])
            #tmp[a[i],:] = h[:, i]*one_hot[a[i]]
            #w2_grads.append(tmp) 
            
            #dhidden = np.dot(dscores, self.model['W2']).T # nhidden x 1
            #dhidden[h[:, i] <= 0] = 0 # RELU backprop
            
            # On W1
            #tmp_2 = np.zeros_like(self.model['W1'])
            #tmp_2 = np.dot(dhidden, )
            # 
            
            
        #for i in range(npar):
        #    predicted_reward.append(prediction[a[i],i])
        #    loss_obs.append(np.sqrt((prediction[a[i], i] - y[i])**2))
            
        #predicted_reward = np.array(predicted_reward) ### 
        #loss_obs = np.array(loss_obs)
        #tmp_loss = np.linalg.norm(loss_obs)
        #loss = tmp_loss/(np.sqrt(npar))
        ###### dscroes as the one hot encoding 
        #dscores = np.zeros_like(prediction)
        
        # Updating Gradients
        #for i in range(npar):
        #    dscores[a[i], i] = 1
        #    action = a[i]
        #    self.grad['W2'][action,:] += h[:,i].T
        
        #self.grad['W2'] = self.grad['W2']/(2*tmp_loss*np.sqrt(npar))
        
        
        
        #for i in range(npar):
        #    action = a[i]
        #    self.grad['W1'] += (1/np.sqrt())
        
        
        
        #self.model['W1'] -= learning_rate*self.grad['W1']
        #self.model['W2'] -= learning_rate*self.grad['W2']
        
        return loss
    
    
    def rmsprop(self, learning_rate, decay_rate): 
        """ Perform model updates from the gradients using RMSprop"""
        for k in self.model:
            g = self.grad[k]
            self.gradsq[k] = decay_rate * self.gradsq[k] + (1 - decay_rate) * g*g
            self.model[k] -= learning_rate * g / (np.sqrt(self.gradsq[k]) + 1e-5)
            self.grad[k].fill(0.0)

## 2. Implement $\epsilon$-Greedy Policy

An $\epsilon$-Greedy policy should:
* with probability $\epsilon$ take a uniformly-random action.
* otherwise choose the best action according to the estimator from the given state.

The function below should implement this policy. It should return a matrix A of size (nactions, npar) such that A[i,j] is the probability of taking action i on input j. The probabilities of non-optimal actions should be $\epsilon/{\rm nactions}$ and the probability of the best action should be $1-\epsilon+\epsilon/{\rm nactions}$.

Since the function processes batches of states, the input <code>state</code> is a <code>ninputs x npar</code> matrix, and the returned value should be a <code>nactions x npar</code> matrix. 

In [6]:
def policy(estimator, state, epsilon):
    """ Take an estimator and state and predict the best action.
    For each input state, return a vector of action probabilities according to an epsilon-greedy policy"""
    ##################################################################################
    ##                                                                              ##
    ## TODO: Implement an epsilon-greedy policy                                     ##
    ##       estimator: is the estimator to use (instance of Estimator)             ##
    ##       state is an (ninputs x npar) state matrix                              ##
    ##       epsilon is the scalar policy parameter                                 ##
    ## return: an (nactions x npar) matrix A where A[i,j] is the probability of     ##
    ##       taking action i on input j.                                            ##
    ##                                                                              ##
    ## Use the definition of epsilon-greedy from the cell above.                    ##
    ##                                                                              ##
    ##################################################################################
    act_val, h = estimator.predict(state) # act_val: naction x npar
    #print(act_val)
    #print(act_val.shape)
    nactions, npar = act_val.shape
    opt_action = np.argmax(act_val, axis=0)
    #print(opt_action)
    A = np.zeros((nactions, npar)) 
    A += epsilon/nactions # the exploration part
    for i in range(npar):
        #print(i)
        A[opt_action[i], i] = 1 - epsilon + (epsilon/nactions)
    return A

In [7]:
state = np.random.random((nfeats*nwindow, npar))
q_estimator1 = Estimator(nfeats*nwindow, nhidden, nactions)
policy(q_estimator1, state, 0.1)

<type 'numpy.ndarray'>


array([[ 0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.95,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05],
       [ 0.95,  0.95,  0.95,  0.95,  0.95,  0.95,  0.95,  0.95,  0.05,
         0.95,  0.95,  0.95,  0.95,  0.95,  0.95,  0.95]])

In [8]:
state.shape

(10, 16)

This routine copies the state of one estimator into another. Its used to update the target estimator from the Q-estimator.

In [9]:
def update_estimator(to_estimator, from_estimator, window, istep):
    """ every <window> steps, Copy model state from from_estimator into to_estimator"""
    if (istep % window == 0):
        for k in from_estimator.model:
            np.copyto(to_estimator.model[k], from_estimator.model[k])

## 3. Implement "Asynchronous Threads"

Don't try that in Python!! Actually all we do here is create an array of environments and advance each one a random number of steps, using random actions at each step. Later on we will make *synchronous* updates to all the environments, but the environments (and their gradient updates) should remain uncorrelated. This serves the same goal as asynchronous updates in paper [2], or experience replay in paper [1].

In [10]:
import random

In [11]:
block_reward = np.zeros((16), dtype=float);
total_epochs = np.zeros((16), dtype=float);

In [12]:
block_reward[0]

0.0

In [13]:
print block_reward.shape, total_epochs.shape

(16,) (16,)


In [14]:
# Create estimators
q_estimator = Estimator(nfeats*nwindow, nhidden, nactions)
target_estimator = Estimator(nfeats*nwindow, nhidden, nactions)

# The epsilon and learning rate decay schedules
epsilons = np.linspace(epsilon_start, epsilon_end, neps)
learning_rates = np.linspace(learning_rate, lr_end, nlr)



In [15]:
nfeats * nwindow

10

In [16]:
import gym

env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        print(env.action_space)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done: 
            print("Episode finished after {} timesteps".format(t+1))
            break

[2016-11-21 22:14:03,847] Making new env: CartPole-v0


[ 0.03324417  0.0340781  -0.03300431 -0.01475731]
Discrete(2)
[ 0.03392573 -0.16055537 -0.03329945  0.26733238]
Discrete(2)
[ 0.03071462  0.03502561 -0.0279528  -0.03566467]
Discrete(2)
[ 0.03141514  0.23053703 -0.0286661  -0.33703429]
Discrete(2)
[ 0.03602588  0.03583449 -0.03540678 -0.05352715]
Discrete(2)
[ 0.03674257  0.23144577 -0.03647733 -0.35716759]
Discrete(2)
[ 0.04137148  0.42706682 -0.04362068 -0.66112592]
Discrete(2)
[ 0.04991282  0.62276774 -0.0568432  -0.96721847]
Discrete(2)
[ 0.06236817  0.81860504 -0.07618757 -1.27720288]
Discrete(2)
[ 0.07874027  1.01461119 -0.10173162 -1.5927367 ]
Discrete(2)
[ 0.0990325   1.21078269 -0.13358636 -1.91533012]
Discrete(2)
[ 0.12324815  1.40706629 -0.17189296 -2.24629005]
Discrete(2)
Episode finished after 12 timesteps
[ 0.01615801  0.04579409  0.04172871  0.01276206]
Discrete(2)
[ 0.01707389 -0.14990069  0.04198395  0.31831338]
Discrete(2)
[ 0.01407588  0.04459893  0.04835022  0.03916069]
Discrete(2)
[ 0.01496786 -0.15118182  0.049133

In [17]:
envs[i].reset().shape

NameError: name 'envs' is not defined

In [19]:
observation_prev = np.zeros((4))
observation_prev.shape

(4,)

In [20]:
# Initialize the games
print("Initializing games..."); sys.stdout.flush()
envs = np.empty(npar, dtype=object)
state = np.zeros([nfeats * nwindow, npar], dtype=float)
rewards = np.zeros([npar], dtype=float)
dones = np.empty(npar, dtype=int)
actions = np.zeros([npar], dtype=int)


for i in range(npar):
    print('agent', i)
    envs[i] = gym.make(game_type)
    ##################################################################################
    ##                                                                              ##
    ## TODO: Advance each environment by a random number of steps, where the number ##
    ##       of steps is sampled uniformly from [nwindow, init_moves].              ##
    ##       Use random steps to advance.                                           ## 
    ##                                                                              ##
    ## Update the total reward and total epochs variables as you go.                ##
    ## If an environment returns done=True, reset it and increment the epoch count. ##
    ##                                                                              ##
    ##################################################################################
    length_game = random.randint(nwindow, init_moves)
    #print(state[:,i].shape)
    #print(envs[i].reset().shape)
    observation_prev = np.zeros((4)) # initiailzing frame t-2
    observation = envs[i].reset()
    for t in range(length_game):
        envs[i].render()
        actions[i] = envs[i].action_space.sample()
        #print(actions[i])
        observation_prev = observation
        observation, rewards[i], dones[i], _ = envs[i].step(actions[i])
        #print(rewards[i])
        block_reward[i] += rewards[i]
        total_epochs[i] += 1
        if dones[i]:
            print("Episode finished after {} timesteps".format(t+1))
            block_reward[i] = 0.0 # resettig total reward
            envs[i].reset()
    rewards[i] = block_reward[i]
    state[:, i] = np.hstack([observation, float(1), observation_prev, float(1)])
    
    
    
    
    
       

Initializing games...


[2016-11-21 22:14:30,270] Making new env: CartPole-v0


('agent', 0)
Episode finished after 20 timesteps
Episode finished after 32 timesteps
Episode finished after 65 timesteps
Episode finished after 84 timesteps
Episode finished after 100 timesteps
Episode finished after 151 timesteps
Episode finished after 177 timesteps
Episode finished after 188 timesteps
Episode finished after 210 timesteps
Episode finished after 222 timesteps
Episode finished after 277 timesteps
Episode finished after 294 timesteps
Episode finished after 322 timesteps
Episode finished after 332 timesteps
Episode finished after 347 timesteps
Episode finished after 366 timesteps
Episode finished after 423 timesteps
Episode finished after 441 timesteps
Episode finished after 453 timesteps
Episode finished after 470 timesteps
Episode finished after 489 timesteps
Episode finished after 498 timesteps
Episode finished after 530 timesteps
Episode finished after 546 timesteps
Episode finished after 582 timesteps
Episode finished after 601 timesteps
Episode finished after 616 ti

[2016-11-21 22:14:47,225] Making new env: CartPole-v0


Episode finished after 1005 timesteps
('agent', 1)
Episode finished after 33 timesteps
Episode finished after 54 timesteps
Episode finished after 65 timesteps
Episode finished after 83 timesteps
Episode finished after 97 timesteps
Episode finished after 113 timesteps
Episode finished after 142 timesteps
Episode finished after 166 timesteps
Episode finished after 178 timesteps
Episode finished after 189 timesteps
Episode finished after 205 timesteps
Episode finished after 233 timesteps
Episode finished after 245 timesteps
Episode finished after 266 timesteps
Episode finished after 332 timesteps
Episode finished after 359 timesteps
Episode finished after 379 timesteps
Episode finished after 394 timesteps
Episode finished after 417 timesteps
Episode finished after 445 timesteps
Episode finished after 456 timesteps
Episode finished after 464 timesteps
Episode finished after 478 timesteps
Episode finished after 507 timesteps
Episode finished after 554 timesteps
Episode finished after 569 ti

...

('agent', 13)
Episode finished after 10 timesteps
Episode finished after 50 timesteps
Episode finished after 62 timesteps
Episode finished after 102 timesteps
Episode finished after 116 timesteps
Episode finished after 159 timesteps
Episode finished after 173 timesteps
Episode finished after 192 timesteps
Episode finished after 207 timesteps
Episode finished after 224 timesteps
Episode finished after 240 timesteps
Episode finished after 261 timesteps
Episode finished after 286 timesteps
Episode finished after 311 timesteps
Episode finished after 331 timesteps
Episode finished after 347 timesteps
Episode finished after 368 timesteps
Episode finished after 385 timesteps
Episode finished after 418 timesteps
Episode finished after 435 timesteps
Episode finished after 460 timesteps
Episode finished after 476 timesteps
Episode finished after 500 timesteps
Episode finished after 546 timesteps
Episode finished after 563 timesteps
Episode finished after 581 timesteps
Episode finished after 605 

[2016-11-21 22:17:35,091] Making new env: CartPole-v0


Episode finished after 1898 timesteps
('agent', 14)
Episode finished after 23 timesteps
Episode finished after 44 timesteps
Episode finished after 64 timesteps


[2016-11-21 22:17:36,724] Making new env: CartPole-v0


Episode finished after 88 timesteps
Episode finished after 104 timesteps
Episode finished after 115 timesteps
Episode finished after 127 timesteps
Episode finished after 147 timesteps
Episode finished after 180 timesteps
Episode finished after 232 timesteps
Episode finished after 245 timesteps
Episode finished after 270 timesteps
Episode finished after 297 timesteps
Episode finished after 314 timesteps
Episode finished after 325 timesteps
Episode finished after 346 timesteps
Episode finished after 369 timesteps
Episode finished after 386 timesteps
Episode finished after 405 timesteps
Episode finished after 436 timesteps
Episode finished after 453 timesteps
Episode finished after 463 timesteps
Episode finished after 478 timesteps
Episode finished after 491 timesteps
Episode finished after 517 timesteps
Episode finished after 528 timesteps
('agent', 15)
Episode finished after 13 timesteps
Episode finished after 33 timesteps
Episode finished after 51 timesteps
Episode finished after 65 ti

In [21]:
Block_Reward = np.sum(block_reward)
Block_Reward

203.0

In [22]:
state[:, 5]

array([ 0.01409005,  0.16856855,  0.0179468 , -0.31191618,  1.        ,
        0.01461573, -0.02628404,  0.01844902, -0.02511075,  1.        ])

## 4. Implement Deep Q-Learning
In this cell you actually implement the algorithm. We've given you comments to define all the steps. You should also add book-keeping steps to keep track of the loss, reward and number of epochs (where env.step() returns done = true). 

In [23]:
env_reward = np

In [24]:
t0 = time.time()
block_loss = 0.0
last_epochs=0
#epsilon = 0.2

y_actual = np.zeros((npar), dtype=float)
a_actual = np.zeros((npar), dtype=float)
print(a_actual)

for istep in np.arange(nsteps): 
    if (render): envs[0].render()
  
    #########################################################################
    ## TODO: Implement Q-Learning                                          ##
    ##                                                                     ##
    ## At high level, your code should:                                    ##
    ## * Update epsilon and learning rate.                                 ##
    ## * Update target estimator from Q-estimator if needed.               ##
    ## * Get the next action probabilities for the minibatch by running    ##
    ##   the policy on the current state with the Q-estimator.             ##
    ## * Then for each environment:                                        ##
    ##     ** Pick an action according to the action probabilities.        ##
    ##     ** Step in the gym with that action.                            ##
    ##     ** Process the observation and concat it to the last nwindow-1  ##
    ##        processed observations to form a new state.                  ##
    ## Then for all environments (vectorized):                             ##
    ## * Predict Q-scores for the new state using the target estimator.    ##
    ## * Compute new expected rewards using those Q-scores.                ##
    ## * Using those expected rewards as a target, compute gradients and   ##
    ##   update the Q-estimator.                                           ##
    ## * Step to the new state.                                            ##
    ##                                                                     ##
    #########################################################################
    indx = int(istep/(nsteps/(1.0*neps)))
    epsilon = epsilons[indx]
    lr = learning_rates[indx]
    update_estimator(q_estimator, target_estimator, nwindow, istep)
    rew, _ = q_estimator.predict(state)
    A = policy(q_estimator, state, epsilon)
    for j in range(npar):
        # ------- Choosing the action using the Greedy Policy ---------- #
        action_prob = A[:, j]
        sampling_prob = np.random.uniform()
        action = int(sampling_prob > action_prob[0])
        #print(action)
        #print(a_actual)
        a_actual[j] = action
        state[5:9, j] = state[0:4, j]
        state[0:4, j], rewards[j], dones[j], _ = envs[j].step(action)
        if dones[j]:
            rewards[j] = 0
        y_actual[j] = rewards[j]
    q_estimator.gradient(state, a_actual, y_actual) 
    q_estimator.rmsprop()    
        
        
    Block_Reward += np.sum(rewards)
    t = time.time() - t0
    if (istep % printsteps == 0):     
        print("step {:0d}, time {:.1f}, loss {:.8f}, epochs {:0d}, reward/epoch {:.5f}".format(
                istep, t, block_loss/printsteps, total_epochs, Block_Reward/np.maximum(1,total_epochs-last_epochs)))
        last_epochs = total_epochs
        Block_Reward = 0.0
        block_loss = 0.0

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


/Users/kevinli/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:66: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/kevinli/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:60: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.
/Users/kevinli/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:67: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/kevinli/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:69: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ValueError: operands could not be broadcast together with shapes (200,) (16,) (200,) 

In [115]:
i = 5
state[:, 5]

array([ 0.23530939,  0.55745888,  0.92382342,  0.39634419,  0.28210271,
        0.92657936,  0.41145885,  0.75122915,  0.86642856,  0.56349944])

In [114]:
state[5:9, 5]

array([ 0.21600844,  0.25735466,  0.86689874,  0.91900546])

Let's save the model now. 

In [ ]:
pickle.dump(q_estimator.model, open("cartpole_q_estimator.p", "wb"))


You can reload the model later if needed:

In [ ]:
test_estimator = Estimator(nfeats*nwindow, nhidden, nactions)
test_estimator.model = pickle.load(open("cartpole_q_estimator.p", "rb"))

And animate the model's performance. 

In [ ]:
state0 = state[:,0]
for i in np.arange(200):
    envs[0].render()
    preds = test_estimator.predict(state0)
    iaction = np.argmax(preds)
    obs, _, done0, _ = envs[0].step(VALID_ACTIONS[iaction])
    state0 = np.concatenate((state0[nfeats:], preprocess(obs)))
    if (done0): envs[0].reset()
    

So there we have it. Simple 1-step Q-Learning can solve easy problems very fast. Note that environments that produce images will be much slower to train on than environments (like CartPole) which return an observation of the state of the system. But this model can still train on those image-based games - like Atari games. It will take hours-days however. It you try training on visual environments, we recommend you run the most expensive step - rmsprop - less often (e.g. every 10 iterations). This gives about a 3x speedup. 

## Optional
Do **one** of the following tasks:
* Adapt the DQN algorithm to another environment - it can use direct state observations.  Call <code>env.get_action_meanings()</code> to find out what actions are allowed. Summarize training performance: your final average reward/epoch, the number of steps required to train, and any modifications to the model or its parameters that you made.
* Try smarter schedules for epsilon and learning rate. Rewards for CartPole increase very sharply (several orders of magnitude) with better policies, especially as epsilon --> 0. Gradients will also change drastically, so the initial learning rate is probably not good later on. Try schedules for decreasing epsilon that allow the model to better adapt. Try other learning rate schedules, or setting learning rate based on average reward. 
* Try a fancier model. e.g. add another hidden layer, or try sigmoid non-linearities.